In [1]:
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import traceback
import logging
from datetime import date
from datetime import datetime

In [2]:
service = Service(executable_path=r"C:\Users\chira\Coding ninjas Data Science\BeautiFul Soup\Selenium\chrome driver\chromedriver_win32\chromedriver.exe")
driver = webdriver.Chrome(service = service)

In [3]:
wait = WebDriverWait(driver,10)

## function to go to a url

In [4]:
def visit_url(driver_,url):
    driver_.get(url)

## Function to login to Instagram


In [5]:
def insta_login(driver,uname,passw):
    
    driver.get('https://www.instagram.com')
    try:
        name_field = wait.until(EC.presence_of_element_located((By.NAME,"username")))
        pass_field = wait.until(EC.presence_of_element_located((By.NAME,"password")))
        name_field.send_keys(uname)
        pass_field.send_keys(passw)
        pass_field.submit()
    except Exception as e:
        logging.error(traceback.format_exc())
    
    
    

In [6]:
# visit_url(driver,'https://www.instagram.com')
# time.sleep(5)
insta_login(driver,'trunksgeta','godtrunks')

# Searching "food" and finding Top 5 handles with most followers out of top 10 handles in search result. Finding Number of posts in past three days by those 5 handles, and plotting this data.

### Function to search a word and return the top n handles, present in search result

In [7]:
def search_and_return(word,n,driver):
    
    driver.refresh()
    time.sleep(5)
    home = driver.find_element(By.CLASS_NAME,'_aacp')
    home.click()
    handles = []
    try:
        side_buttons = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME,'_aacp')))
        search_button = side_buttons[1]
        search_button.click()
        
        try:
            search_box = wait.until(EC.presence_of_element_located((By.CLASS_NAME,'_aauy')))
            search_box.send_keys(word)
            
            time.sleep(5)
            
            try:
                results = wait.until(EC.presence_of_all_elements_located((By.XPATH,'//div[@class="_abm4"]//div[contains(@class,"_aad6") and contains(@class,"_aaco")]'))) 
                i = 0
                for r in results:
                    if r.text == '':
                        continue
                    if r.text[0] == '#':
                        continue
                    i += 1
                    handles.append(r.text)
                    if i==n :
                        break
                return handles
            except Exception as e:
                logging.error(traceback.format_exc())      
        except Exception as e:
            logging.error(traceback.format_exc())
    except Exception as e:
        logging.error(traceback.format_exc())
    
    
    

## Finding the Top 10 Handles when 'food' is searched

In [10]:
search_results = search_and_return('food',10,driver)

In [11]:
for i in range(len(search_results)):
    print(i + 1,"---->",search_results[i])

1 ----> foodiesdelhite
2 ----> foodisnirvana
3 ----> foodpandits
4 ----> foodieveggie
5 ----> foodbellydelites
6 ----> food_and_makeup_lover
7 ----> food
8 ----> foodtalkindia
9 ----> punjabi_food_factory
10 ----> thefoodsquare_guna


## Function to go to profile page of a particular handle

In [12]:
def go_to_profile(handle,driver):
    
    driver.refresh()
    time.sleep(5)
    home = driver.find_element(By.CLASS_NAME,'_aacp')
    home.click()
    time.sleep(5)
    try:
        side_buttons = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME,'_aacp')))
        search_button = side_buttons[1]
        search_button.click()
        
        try:
            search_box = wait.until(EC.presence_of_element_located((By.CLASS_NAME,'_aauy')))
            search_box.send_keys(handle)
            
#             time.sleep(10)
            
            try:
                results = wait.until(EC.presence_of_all_elements_located((By.XPATH,'//div[@class="_abm4"]//div[contains(@class,"_aad6") and contains(@class,"_aaco")]'))) 
                found = False
                for r in results:
                    if r.text == '':
                        continue
                    if r.text[0] == '#':
                        continue
                    if r.text == handle:
                        found = True
                        print("Handle found, going to profile page")
                        r.click()
                        break
                    
                if found == False:
                    print("Required Handle not found")
            except Exception as e:
                logging.error(traceback.format_exc())      
        except Exception as e:
            logging.error(traceback.format_exc())
    except Exception as e:
        logging.error(traceback.format_exc())
    
    
    
    

### Function to remove all occurences of a character from a word

In [13]:
def removechar(word, ch): #removes character ch from word
    i = 0
    while(i < len(word)):
     
        if(word[i] == ch):
            word = word[:i] + word[i+1:]
            i -= 1
        i += 1
    return word

### Function to get number of Followers and Following

In [15]:
def get_count_ff(driver): #Find the number of followers and following from profile page on which user is currently on
    
    pfF = driver.find_elements(By.CLASS_NAME,"_ac2a")
    if len(pfF)<3 :
        return 0,0
    
    p = pfF[0].text
    f = pfF[1].text
    F = pfF[2].text
    
#     print(f)
#     print(F)
    f = removechar(f,',')
    F = removechar(F,',')
#     print("___________________")
#     print(f)
#     print(F)
    
    if f[-1] == 'K':
        f = float(f[0:-1:1])
        f = f*1000
    elif f[-1] == 'M':
        f = float(f[0:-1:1])
        f = f*1000000
    else:
        f = int(f)
    
    if F[-1] == 'K':
        F = float(F[0:-1:1])
        F = F*1000
    elif F[-1] == 'M':
        F = float(F[0:-1:1])
        F = F*1000000
    else:
        F = float(F)
    
    return f,F
    
    

In [16]:
followers = []
for i in range(0,len(search_results)):
    print(" trying for going to profile of ",search_results[i])
    go_to_profile(search_results[i],driver) #go to profile page
    time.sleep(5) #wait for page to load
    f,F = get_count_ff(driver) # find number of followers and following
    followers.append((search_results[i],f)) # appending account name and number of followers in as a tuple in a list
    print(search_results[i]," has ",f," followers. ")

 trying for going to profile of  foodiesdelhite
Handle found, going to profile page
foodiesdelhite  has  23400.0  followers. 
 trying for going to profile of  foodisnirvana
Handle found, going to profile page
foodisnirvana  has  20200.0  followers. 
 trying for going to profile of  foodpandits
Handle found, going to profile page
foodpandits  has  183000.0  followers. 
 trying for going to profile of  foodieveggie
Handle found, going to profile page
foodieveggie  has  6745  followers. 
 trying for going to profile of  foodbellydelites
Handle found, going to profile page
foodbellydelites  has  7176  followers. 
 trying for going to profile of  food_and_makeup_lover
Handle found, going to profile page
food_and_makeup_lover  has  13600.0  followers. 
 trying for going to profile of  food
Handle found, going to profile page
food  has  182000.0  followers. 
 trying for going to profile of  foodtalkindia
Handle found, going to profile page
foodtalkindia  has  0  followers. 
 trying for going 

In [17]:
for i,j in followers:
    print(i,j)

foodiesdelhite 23400.0
foodisnirvana 20200.0
foodpandits 183000.0
foodieveggie 6745
foodbellydelites 7176
food_and_makeup_lover 13600.0
food 182000.0
foodtalkindia 0
punjabi_food_factory 2572
thefoodsquare_guna 1746


In [18]:
followers.sort(key = lambda x:x[1],reverse = True) #Sorting the list made by number of followers in decreasing order

In [19]:
for i,j in followers:
    print(i,j)

foodpandits 183000.0
food 182000.0
foodiesdelhite 23400.0
foodisnirvana 20200.0
food_and_makeup_lover 13600.0
foodbellydelites 7176
foodieveggie 6745
punjabi_food_factory 2572
thefoodsquare_guna 1746
foodtalkindia 0


In [20]:
top_5_profiles = followers[:5]
print("The 5 profiles with the most number of followers : ")
for i in top_5_profiles:
    print(i[0])

The 5 profiles with the most number of followers : 
foodpandits
food
foodiesdelhite
foodisnirvana
food_and_makeup_lover


### Function to find the number of posts in past n days

In [ ]:
def find_number_of_posts(driver): # Find the number of posts by the account that driver is currenly on
    
    try:
        posts = wait.until(EC.get_all_elements_located((By.CLASS_NAME,"_aa")))
    

In [23]:
date_of_post = driver.find_element(By.CLASS_NAME,"_aaqe")

In [24]:
dt = date_of_post.get_attribute('datetime')

In [25]:
dt

'2023-02-04T13:04:44.000Z'

In [26]:
type(dt)

str

In [29]:
todays_date = date.today()

In [30]:
dt = dt[0:10]

In [31]:
dt

'2023-02-04'

In [44]:
post_date = datetime.strptime(dt,"%Y-%m-%d")
post_date = datetime.date(post_date)

In [45]:
post_date

datetime.date(2023, 2, 4)

In [46]:
diff = todays_date - post_date

In [48]:
diff.days

4

In [49]:
diff.seconds

0

In [50]:
diff.days

4